# Exploração da base de dados do RAIS

Neste notebook, iremos carregar e realizar exploração de dados.

In [ ]:
import os
import pandas as pd
import requests
import py7zr
from ftplib import FTP
import numpy as np
from tqdm import tqdm
import time

# --- Configuração ---

# Diretório para baixar/extrair os dados brutos
WORKING_DIRECTORY = r"C:/Users/john-/OneDrive - Universidade Federal da Paraíba/Área de Trabalho\Artigo - Estatística/projeto-evasao/data/raw/rais"
os.makedirs(WORKING_DIRECTORY, exist_ok=True)
os.chdir(WORKING_DIRECTORY)

# Diretório para salvar os dados processados (resultados)
PROCESSED_DIRECTORY = r"C:/Users/john-/OneDrive - Universidade Federal da Paraíba/Área de Trabalho/Artigo - Estatística/projeto-evasao/data/processed"
os.makedirs(PROCESSED_DIRECTORY, exist_ok=True)

YEAR = 2023
RAIS_BASES = [
    "RAIS_VINC_PUB_SP",
    "RAIS_VINC_PUB_CENTRO_OESTE",
    "RAIS_VINC_PUB_MG_ES_RJ",
    "RAIS_VINC_PUB_NORDESTE",
    "RAIS_VINC_PUB_NORTE",
    "RAIS_VINC_PUB_SUL",
]

SELECTED_VARIABLES = [
    "CBO Ocupação 2002",
    "Vínculo Ativo 31/12",
    "Vl Remun Média Nom",
    "Município",
]

COLUMN_RENAME_MAP = {
    "CBO Ocupação 2002": "cbo_ocup",
    "Vínculo Ativo 31/12": "vic_ativ",
    "Vl Remun Média Nom": "rem_med",
    "Município": "mun",
}

# --- Funções Auxiliares ---

def download_file(url, dest_path, retries=10, delay=60):
    """Baixa um arquivo de uma URL (suporta HTTP/HTTPS e FTP) com tentativas."""
    for i in range(retries):
        try:
            if url.startswith("ftp://"):
                from urllib.parse import urlparse
                parsed_url = urlparse(url)
                ftp_host = parsed_url.hostname
                ftp_path = parsed_url.path.lstrip('/')

                tqdm.write(f"  Tentativa {i+1}: Baixando {os.path.basename(dest_path)} via FTP...")
                with FTP(ftp_host) as ftp:
                    ftp.login()
                    with open(dest_path, 'wb') as fp:
                        ftp.retrbinary(f'RETR {ftp_path}', fp.write)
                tqdm.write(f"  Arquivo {os.path.basename(dest_path)} baixado via FTP.")
                return True
            else:
                tqdm.write(f"  Tentativa {i+1}: Baixando {os.path.basename(dest_path)} via HTTP/HTTPS...")
                response = requests.get(url, stream=True, timeout=500)
                response.raise_for_status()
                with open(dest_path, "wb") as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        f.write(chunk)
                tqdm.write(f"  Arquivo {os.path.basename(dest_path)} baixado via HTTP/HTTPS.")
                return True
        except Exception as e:
            tqdm.write(f"  Tentativa {i+1} falhou ao baixar {os.path.basename(dest_path)}: {e}")
            if i < retries - 1:
                tqdm.write(f"  Tentando novamente em {delay} segundos...")
                time.sleep(delay)
            else:
                tqdm.write(f"  Falha ao baixar {os.path.basename(dest_path)} após {retries} tentativas.")
                return False

def extract_7z(file_path, dest_dir):
    """Extrai um arquivo 7z."""
    try:
        tqdm.write(f"  Extraindo {os.path.basename(file_path)}...")
        with py7zr.SevenZipFile(file_path, mode="r") as z:
            z.extractall(path=dest_dir)
        tqdm.write(f"  Arquivo {os.path.basename(file_path)} extraído.")
        return True
    except Exception as e:
        tqdm.write(f"  Erro ao extrair {file_path}: {e}")
        return False

# --- Aquisição e Pré-processamento de Dados ---

def acquire_and_preprocess_data():
    all_data_frames = []

    for base_name in tqdm(RAIS_BASES, desc=f"Processando bases da RAIS para o ano {YEAR}"):
        file_7z = f"{base_name}.7z"
        file_txt = f"{base_name}.txt"
        ftp_path = f"ftp://ftp.mtps.gov.br/pdet/microdados/RAIS/{YEAR}/{file_7z}"
        rda_file = f"{base_name}.pkl"

        if not download_file(ftp_path, file_7z):
            continue

        if not extract_7z(file_7z, "."):
            if os.path.exists(file_7z):
                os.remove(file_7z)
            continue

        try:
            df = pd.read_csv(
                file_txt,
                sep=";",
                usecols=SELECTED_VARIABLES,
                encoding="latin-1",
                dtype={"CBO Ocupação 2002": str},
            )
            df.to_pickle(rda_file)
            all_data_frames.append(df)
            tqdm.write(f"  Dados de {base_name} carregados e salvos em {rda_file}")
        except Exception as e:
            tqdm.write(f"  Erro ao carregar ou salvar {file_txt}: {e}")
        finally:
            if os.path.exists(file_7z):
                os.remove(file_7z)
            if os.path.exists(file_txt):
                os.remove(file_txt)

    if not all_data_frames:
        tqdm.write("Nenhum data frame foi carregado com sucesso.")
        return None

    tqdm.write("\nCombinando todos os dados regionais...")
    x = pd.concat(all_data_frames, ignore_index=True)
    tqdm.write("Todos os dados regionais combinados.")

    tqdm.write("Renomeando colunas...")
    x = x.rename(columns={k: v for k, v in COLUMN_RENAME_MAP.items() if k in x.columns})
    tqdm.write("Colunas renomeadas.")

    tqdm.write("Convertendo 'rem_med' para numérico...")
    x["rem_med"] = x["rem_med"].str.replace(",", ".").astype(float)
    tqdm.write("'rem_med' convertido para numérico.")

    tqdm.write("Filtrando dados...")
    filtered_data = x[
        (x["vic_ativ"] == 1)
        & (x["rem_med"].between(220, 100000))
    ].copy()
    tqdm.write("Dados filtrados de acordo com 'Vínculo Ativo 31/12' e 'Vl Remun Média Nom'.")

    return filtered_data

def calculate_mean_salary_by_municipality_and_cbo(df):
    if 'mun' not in df.columns or 'cbo_ocup' not in df.columns or 'rem_med' not in df.columns:
        tqdm.write("DataFrame não contém as colunas 'mun', 'cbo_ocup' ou 'rem_med'. Não é possível calcular a média salarial.")
        return None

    tqdm.write("Calculando a média salarial por município e CBO Ocupação 2002...")
    mean_salary = df.groupby(['mun', 'cbo_ocup'])['rem_med'].mean().reset_index()
    mean_salary.rename(columns={'rem_med': 'media_salarial'}, inplace=True)
    tqdm.write("Média salarial por município e CBO Ocupação 2002 calculada.")
    return mean_salary

# --- Main Execution ---

def main():
    tqdm.write("Iniciando a aquisição e pré-processamento de dados...")
    filtered_data = acquire_and_preprocess_data()

    if filtered_data is None:
        tqdm.write("Não foi possível adquirir ou pré-processar os dados. O cálculo da média salarial não será realizado.")
        return

    if filtered_data.empty:
        tqdm.write("O DataFrame filtrado está vazio após o pré-processamento. Não há dados para calcular a média salarial.")
        return

    mean_salary_df = calculate_mean_salary_by_municipality_and_cbo(filtered_data)

    if mean_salary_df is None:
        tqdm.write("O cálculo da média salarial retornou None (provavelmente devido a colunas ausentes). O arquivo CSV não será salvo.")
    else:
        file_path = os.path.join(PROCESSED_DIRECTORY, f"media_salarial_municipio_cbo_{YEAR}.csv")
        try:
            mean_salary_df.to_csv(file_path, index=False)
            tqdm.write(f"Média salarial por município e CBO salva em '{file_path}'.")
        except Exception as e:
            tqdm.write(f"Erro ao salvar o arquivo CSV: {e}")

    tqdm.write("\nProcesso concluído.")

if __name__ == "__main__":
    main()



Iniciando a aquisição e pré-processamento de dados...


Processando bases da RAIS para o ano 2023:   0%|          | 0/1 [00:00<?, ?it/s]

  Tentativa 1: Baixando RAIS_VINC_PUB_SUL.7z via FTP...


Processando bases da RAIS para o ano 2023:   0%|          | 0/1 [25:37<?, ?it/s]

  Arquivo RAIS_VINC_PUB_SUL.7z baixado via FTP.
  Extraindo RAIS_VINC_PUB_SUL.7z...


...

  Arquivo RAIS_VINC_PUB_SUL.7z extraído.


Processando bases da RAIS para o ano 2023:   0%|          | 0/1 [27:43<?, ?it/s]

  Dados de RAIS_VINC_PUB_SUL carregados e salvos em RAIS_VINC_PUB_SUL.pkl


Processando bases da RAIS para o ano 2023: 100%|██████████| 1/1 [27:43<00:00, 1663.88s/it]



Combinando todos os dados regionais...
Todos os dados regionais combinados.
Renomeando colunas...
Colunas renomeadas.
Convertendo 'rem_med' para numérico...
'rem_med' convertido para numérico.
Filtrando dados...
Dados filtrados de acordo com 'Vínculo Ativo 31/12' e 'Vl Remun Média Nom'.
Calculando a média salarial por município e CBO Ocupação 2002...
Média salarial por município e CBO Ocupação 2002 calculada.
Média salarial por município e CBO salva em 'C:/Users/john-/OneDrive - Universidade Federal da Paraíba/Área de Trabalho/Artigo - Estatística/projeto-evasao/data/processed\media_salarial_municipio_cbo_2023.csv'.

Processo concluído.


In [1]:
import pandas as pd
import os

# Definir caminho da pasta
caminho = r"C:\Users\john-\OneDrive - Universidade Federal da Paraíba\Área de Trabalho\Artigo - Estatística\projeto-evasao\data\processed"

# Ler os arquivos
resultado_match = pd.read_excel(os.path.join(caminho, 'resultado_match_cbo_cursos.xlsx'))
media_salarial = pd.read_csv(os.path.join(caminho, 'media_salarial_2018_2023.csv'))

# Padronizar a coluna 'cbo_ocup' nas duas bases
for df in [resultado_match, media_salarial]:
    df['cbo_ocup'] = df['cbo_ocup'].astype(str).str.strip().str.lower()

# Remover linhas com cbo_ocup ausente
resultado_match = resultado_match.dropna(subset=['cbo_ocup'])
media_salarial = media_salarial.dropna(subset=['cbo_ocup'])

# Checagem de CBOs únicos nas duas bases
cbo_cursos = set(resultado_match['cbo_ocup'].unique())
cbo_salarios = set(media_salarial['cbo_ocup'].unique())

# Interseção (CBOs que existem nas duas bases)
cbo_comum = cbo_cursos.intersection(cbo_salarios)

print("🔍 Checagem de CBOs:")
print(f"- Total de CBOs únicos na base de cursos: {len(cbo_cursos)}")
print(f"- Total de CBOs únicos na base salarial: {len(cbo_salarios)}")
print(f"- Total de CBOs que deram match: {len(cbo_comum)}")
print(f"- CBOs sem correspondência na base salarial: {len(cbo_cursos - cbo_salarios)}")
print(f"- CBOs sem correspondência na base de cursos: {len(cbo_salarios - cbo_cursos)}")

# (Opcional) Listar quais ficaram de fora
print("\n📄 CBOs da base de cursos SEM salário:")
print(cbo_cursos - cbo_salarios)

print("\n📄 CBOs da base salarial SEM curso:")
print(cbo_salarios - cbo_cursos)

# Realizar o merge
media_salario_2018_2023 = pd.merge(
    resultado_match,
    media_salarial,
    on='cbo_ocup',
    how='inner'  # 'left' se quiser manter todos os cursos
)

# Salvar o resultado
media_salario_2018_2023.to_csv(
    os.path.join(caminho, 'media_salario_2018_2023.csv'),
    index=False,
    encoding='utf-8-sig'
)

print("\n✅ Merge realizado com sucesso e arquivo salvo!")


🔍 Checagem de CBOs:
- Total de CBOs únicos na base de cursos: 2601
- Total de CBOs únicos na base salarial: 2724
- Total de CBOs que deram match: 2601
- CBOs sem correspondência na base salarial: 0
- CBOs sem correspondência na base de cursos: 123

📄 CBOs da base de cursos SEM salário:
set()

📄 CBOs da base salarial SEM curso:
{'142125', '253405', '523120', '111120', '20205', '516910', '142130', '10215', '512105', '512110', '423115', '204110', '842220', '322140', '111105', '111115', '21210', '622710', '212425', '253410', '517130', '20110', '30110', '21205', '111245', '10305', '424205', '111250', '111210', '342560', '512115', '31105', '374155', '30205', '20115', '314835', '324135', '142140', '411055', '214540', '999999', '516155', '781310', '223915', '141820', '10105', '715420', '322255', '314840', '375130', '211220', '141815', '223575', '239445', '314830', '141805', '391145', '354610', '375125', '314825', '225355', '141810', '725030', '111205', '524315', '515320', '226110', '422335', '

In [2]:
import pandas as pd
import os

# Definir caminho da pasta
caminho = r"C:\Users\john-\OneDrive - Universidade Federal da Paraíba\Área de Trabalho\Artigo - Estatística\projeto-evasao\data\processed"

# Ler o arquivo já com merge
media_salario_2018_2023 = pd.read_csv(os.path.join(caminho, 'media_salario_2018_2023.csv'))

# Agrupar por 'mun' e 'CURSOS' e calcular a média
media_salarial_cursos_2018_2023 = media_salario_2018_2023.groupby(['mun', 'CURSOS']).mean(numeric_only=True).reset_index()

# Visualizar o resultado
print(media_salarial_cursos_2018_2023)

# Salvar o resultado em CSV
media_salarial_cursos_2018_2023.to_csv(
    os.path.join(caminho, 'media_salarial_cursos_2018_2023.csv'),
    index=False,
    encoding='utf-8-sig'
)

print("\n✅ Média salarial por município e curso calculada e salva com sucesso!")



C:\Users\john-\AppData\Local\Temp\ipykernel_23532\1783581962.py:8: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  media_salario_2018_2023 = pd.read_csv(os.path.join(caminho, 'media_salario_2018_2023.csv'))


           mun                          CURSOS  media_salarial_media_2018_2023
0       110001                   ADMINISTRAÇÃO                     2823.666662
1       110001           ADMINISTRAÇÃO PÚBLICA                     2898.938086
2       110001                       AGRONOMIA                     1785.029910
3       110001              CIÊNCIAS CONTÁBEIS                     2000.551607
4       110001             CIÊNCIAS ECONÔMICAS                     4599.226889
...        ...                             ...                             ...
225191  999999          ENGENHARIA DE PRODUÇÃO                    16823.255000
225192  999999             ENGENHARIA ELÉTRICA                    26242.800000
225193  999999             ENGENHARIA MECÂNICA                    48916.565000
225194  999999  TECNOLOGIA EM GESTÃO COMERCIAL                    24165.400000
225195  999999                         TURISMO                    56922.450000

[225196 rows x 3 columns]

✅ Média salarial por mun

In [22]:
import pandas as pd
import os
import unicodedata

# =========================
# Função para padronizar texto
# =========================
def padronizar_texto(texto):
    if pd.isna(texto):
        return ''
    texto = str(texto).strip().lower()
    texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8')
    texto = texto.replace('-', ' ').replace('_', ' ')
    texto = ' '.join(texto.split())
    return texto

# =========================
# Caminho
# =========================
caminho = r"C:\Users\john-\OneDrive - Universidade Federal da Paraíba\Área de Trabalho\Artigo - Estatística\projeto-evasao\data\processed"

# =========================
# Leitura dos arquivos
# =========================
ml_iq_2018_2023 = pd.read_csv(os.path.join(caminho, 'ml_iq_2018_2023.csv'))
media_salarial_cursos_2018_2023 = pd.read_csv(os.path.join(caminho, 'media_salarial_cursos_2018_2023.csv'))

# =========================
# Tratamento da coluna CÓDIGO_DO_MUNICÍPIO
# =========================
cod_mun = (
    ml_iq_2018_2023['CÓDIGO_DO_MUNICÍPIO']
    .astype(str)
    .str.replace('.', '', regex=False)
    .str[:-1]
)

# Converte para número, forçando valores inválidos a NaN e usando Int64 para permitir nulos
ml_iq_2018_2023['CÓDIGO_DO_MUNICÍPIO'] = pd.to_numeric(cod_mun, errors='coerce').astype('Int64')

print(f"Valores nulos após tratamento: {ml_iq_2018_2023['CÓDIGO_DO_MUNICÍPIO'].isna().sum()}")

# =========================
# Padronização dos nomes dos cursos
# =========================
ml_iq_2018_2023['ÁREA_DE_AVALIAÇÃO'] = ml_iq_2018_2023['ÁREA_DE_AVALIAÇÃO'].apply(padronizar_texto)
media_salarial_cursos_2018_2023['CURSOS'] = media_salarial_cursos_2018_2023['CURSOS'].apply(padronizar_texto)

# =========================
# Merge dos dataframes
# =========================
df_merged = ml_iq_2018_2023.merge(
    media_salarial_cursos_2018_2023,
    left_on=['CÓDIGO_DO_MUNICÍPIO', 'ÁREA_DE_AVALIAÇÃO'],
    right_on=['mun', 'CURSOS'],
    how='left'
)

# =========================
# Criar coluna 'salario'
# =========================
df_merged['salario'] = df_merged['media_salarial_media_2018_2023']

# =========================
# Salvar o novo dataframe
# =========================
df_merged.to_csv(
    os.path.join(caminho, 'ml_iq_2018_2023.csv'),
    index=False,
    encoding='utf-8-sig'
)

print("\n✅ Merge concluído com sucesso e coluna 'salario' adicionada!")

# =========================
# Verificar combinações sem salário
# =========================
na_salario = df_merged[df_merged['salario'].isna()]

print(f"\n🔎 Total de registros sem salário encontrado: {len(na_salario)}")

combinacoes_sem_match = na_salario[['CÓDIGO_DO_MUNICÍPIO', 'ÁREA_DE_AVALIAÇÃO']].drop_duplicates()

print("\n🚩 Combinações sem correspondência de salário:\n")
print(combinacoes_sem_match)

combinacoes_sem_match.to_csv(
    os.path.join(caminho, 'combinacoes_sem_salario.csv'),
    index=False,
    encoding='utf-8-sig'
)

print("\n✅ Arquivo 'combinacoes_sem_salario.csv' salvo com sucesso!")





Valores nulos após tratamento: 1

✅ Merge concluído com sucesso e coluna 'salario' adicionada!

🔎 Total de registros sem salário encontrado: 54

🚩 Combinações sem correspondência de salário:

      CÓDIGO_DO_MUNICÍPIO                  ÁREA_DE_AVALIAÇÃO
373                431690                     fonoaudiologia
588                261160                     fonoaudiologia
589                292740                     fonoaudiologia
728                330100       tecnologia em design grafico
739                314610  tecnologia em gestao da qualidade
824                172100       tecnologia em gestao publica
865                431490                     fonoaudiologia
896                280030                     fonoaudiologia
952                130120                           nutricao
985                210320                          zootecnia
1027               150140       tecnologia em gestao publica
1029               432050         tecnologia em agronegocios
1176           

In [21]:
import os
import pandas as pd

# Caminho do arquivo
caminho = r"C:\Users\john-\OneDrive - Universidade Federal da Paraíba\Área de Trabalho\Artigo - Estatística\projeto-evasao\data\processed"
arquivo = "ml_iq_2018_2023.csv"

# Carregar o dataframe
df_merged = pd.read_csv(os.path.join(caminho, arquivo))

# Mostrar as colunas atuais para garantir que as que deseja excluir existem
print("Colunas atuais:", df_merged.columns.tolist())

# Excluir as colunas, ignorando erro se não existir
colunas_para_excluir = ['mun', 'CURSOS', 'media_salarial_media_2018_2023', 'salario']
df_merged = df_merged.drop(columns=[col for col in colunas_para_excluir if col in df_merged.columns])

# Salvar o novo dataframe sem essas colunas
df_merged.to_csv(
    os.path.join(caminho, 'ml_iq_2018_2023_sem_colunas.csv'),
    index=False,
    encoding='utf-8-sig'
)

print("Arquivo salvo sem as colunas indicadas.")



Colunas atuais: ['CÓDIGO_DA_IES', 'NOME_DA_IES', 'SIGLA_DA_IES', 'ORGANIZAÇÃO_ACADÊMICA', 'CATEGORIA_ADMINISTRATIVA', 'CÓDIGO_DO_CURSO', 'CÓDIGO_DA_ÁREA', 'ÁREA_DE_AVALIAÇÃO', 'MODALIDADE_DE_ENSINO', 'CÓDIGO_DO_MUNICÍPIO', 'MUNICÍPIO_DO_CURSO', 'SIGLA_DA_UF', 'NOTA_PADRONIZADA_-_IDD', 'NOTA_PADRONIZADA_-_ORGANIZAÇÃO_DIDÁTICO-PEDAGÓGICA', 'NOTA_PADRONIZADA_-_INFRAESTRUTURA_E_INSTALAÇÕES_FÍSICAS', 'NOTA_PADRONIZADA_-_OPORTUNIDADE_DE_AMPLIAÇÃO_DA_FORMAÇÃO', 'NOTA_PADRONIZADA_-_MESTRES', 'NOTA_PADRONIZADA_-_DOUTORES', 'MEDIA_CONCEITO_ENADE_(CONTÍNUO)', 'IDD_(CONTÍNUO)', 'CONCEITO_MÉDIO_DE_GRADUAÇÃO', 'IGC_(CONTÍNUO)', 'TDA', 'TDA_binaria', 'mun', 'CURSOS', 'media_salarial_media_2018_2023', 'salario']
Arquivo salvo sem as colunas indicadas.


In [13]:
import pandas as pd

# Caminho do arquivo original
caminho_arquivo = r"C:\Users\john-\OneDrive - Universidade Federal da Paraíba\Área de Trabalho\Artigo - Estatística\projeto-evasao\data\processed\ml_iq_2018_2023.csv"

# Carregar o DataFrame
ml_iq_2018_2023 = pd.read_csv(caminho_arquivo)

# Garantir que a coluna seja string
ml_iq_2018_2023['CÓDIGO_DO_MUNICÍPIO'] = ml_iq_2018_2023['CÓDIGO_DO_MUNICÍPIO'].astype(str)

# Pegar os 6 primeiros dígitos
ml_iq_2018_2023['CÓDIGO_DO_MUNICÍPIO'] = ml_iq_2018_2023['CÓDIGO_DO_MUNICÍPIO'].str[:6]

# Converter para inteiro
ml_iq_2018_2023['CÓDIGO_DO_MUNICÍPIO'] = ml_iq_2018_2023['CÓDIGO_DO_MUNICÍPIO'].astype(int)

# Salvar em um novo arquivo CSV
novo_caminho = r"C:\Users\john-\OneDrive - Universidade Federal da Paraíba\Área de Trabalho\Artigo - Estatística\projeto-evasao\data\processed\ml_iq_2018_2023_alterado.csv"
ml_iq_2018_2023.to_csv(novo_caminho, index=False, encoding='utf-8-sig')

print("Arquivo salvo em:", novo_caminho)





Arquivo salvo em: C:\Users\john-\OneDrive - Universidade Federal da Paraíba\Área de Trabalho\Artigo - Estatística\projeto-evasao\data\processed\ml_iq_2018_2023_alterado.csv


In [23]:
import os
import pandas as pd

# Caminho do arquivo
caminho = r"C:\Users\john-\OneDrive - Universidade Federal da Paraíba\Área de Trabalho\Artigo - Estatística\projeto-evasao\data\processed"
arquivo = "ml_iq_2018_2023.csv"
caminho_arquivo = os.path.join(caminho, arquivo)

# Carregar o dataframe
df = pd.read_csv(caminho_arquivo)

# Remover linhas com missing values
df_limpo = df.dropna()

# Salvar sobrescrevendo o arquivo original
df_limpo.to_csv(caminho_arquivo, index=False, encoding='utf-8-sig')

print("Arquivo salvo após remoção de missing values.")


Arquivo salvo após remoção de missing values.


In [1]:
import math

# Tentando diretamente (gera overflow)
try:
    resultado = 1 / (1 + math.exp(9995))
    print(resultado)
except OverflowError:
    print("Overflow: resultado é praticamente zero.")


Overflow: resultado é praticamente zero.
